In [1]:
import pandas as pd # pip install pandas
import matplotlib.pyplot as plt # pip install matplotlib
from statsmodels.tsa.seasonal import seasonal_decompose # pip install statsmodels
import yfinance as yf # pip install yfinance

In [ ]:
ticker = yf.Ticker('PETR4.SA')
# Data AAAA/MM/DD
df = ticker.history(interval = '1d', start = '2021-01-01', end = '2024-01-01')
df

In [ ]:
df[['Close']].plot()

In [ ]:
# Preço de fechamento e a tendencia. 
decomposicao = seasonal_decompose(df[['Close']], model= 'additive', period= 30, extrapolate_trend=30)

plt.plot(decomposicao.observed)
plt.plot(decomposicao.trend)

In [ ]:
# Média móvel de 7, 30 e 90 dias.
media_movel_7d = df['Close'].rolling(7).mean()
media_movel_30d = df['Close'].rolling(30).mean()
media_movel_90d = df['Close'].rolling(90).mean()

fix, ax = plt.subplots(figsize=(12,5))
plt.plot(media_movel_7d, 'orange')
plt.plot(media_movel_30d, 'red')
plt.plot(media_movel_90d, 'blue')
plt.plot(df['Close'])

In [ ]:
df2 = df.reset_index()
df2.head()

In [ ]:
df2['year']=df2['Date'].dt.year
df2['month']=df2['Date'].dt.month
df2['day']=df2['Date'].dt.day
df2.head()

In [ ]:
# Rentabilidade 
df2['rentabilidade']=(df2['Close']/df2['Close'].shift()*100)-100
df2.head()

In [ ]:
# Existe um melhor mês para investir na Bolsa?
df2.groupby('month').agg({'rentabilidade':'mean'}).plot(kind='bar', title='Rentabilidade média por mês', figsize=(11,5)).set_ylabel('Variação (%)')
plt.show

In [ ]:
# Existe um melhor dia para investir na Bolsa?
df2.groupby('day').agg({'rentabilidade':'sum'}).plot(kind='bar', title='Dias do mês com maiores rentabilidades', figsize=(11,5)).set_ylabel('Soma da rentabilidade')
plt.show

In [ ]:
# Análise da correlação.
tickers =['ITUB4.SA', 'VALE3.SA', 'BBAS3.SA']

dfs=[]

for t in tickers:
    print(f'Carregando ticker {t}...')
    ticker = yf.Ticker(t)
    aux = ticker.history(interval = '1d', start= '2021-01-01', end= '2024-01-01')
    aux.reset_index(inplace=True)
    aux['ticker']=t
    dfs.append(aux)

In [12]:
def features_dfs(df_):
    df_['year']=df_['Date'].dt.year
    df_['month']=df_['Date'].dt.month
    df_['day']=df_['Date'].dt.day
    df_['rentabilidade']=(df_['Close']/df_['Close'].shift()*100)-100
    

In [13]:
for d in dfs:
    features_dfs(d)

In [ ]:
# Correlação
correlacao = pd.DataFrame()
for d in dfs:
    correlacao[d['ticker'].loc[0]]=d['rentabilidade']

correlacao.head()

In [ ]:
correlacao.corr()

In [ ]:
import seaborn as sns

ax, fig = plt.subplots(figsize=(20,7))
ax = sns.heatmap(correlacao.corr(), annot=True)

In [22]:
import plotly.graph_objs as go
def plotCandStick(df, acao='ticker'):
    tracel = {
        'x': df.index,
        'open': df.Open,
        'close': df.Close,
        'high': df.High,
        'low': df.Low,
        'type': 'candlestick',
        'name': acao,
        'showlegend': False
    }

    data = [tracel]
    layout=go.Layout()
    fig=go.Figure(data=data, layout=layout)
    return fig

In [ ]:
plotCandStick(df2)